# CISC 525 Spring 2020 - Final Exam
You may want to revisit lecture from Mondule 9 on NoSQL database.
I provided instructions on installing necessary software package
to work with MongoDB using Python 3.


In [46]:
from pymongo import MongoClient
#include pprint for readabillity of the 
from pprint import pprint  # pretty print
from random import randint

#change the MongoClient connection string to your MongoDB database instance
client = MongoClient(port=27017)
airline = client.airline

In [47]:
client.list_database_names()

['admin', 'airline', 'config', 'local']

In [48]:
db = client["airline"]

## Importing airline performance data
You **`must`** fix the content of line `1253` in the `airports.csv` file
from: 

```
"DBN","W. H. \"Bud\" Barron ","Dublin","GA","USA",32.56445806,-82.98525556
```

to 

```
"DBN","W. H. Bud Barron ","Dublin","GA","USA",32.56445806,-82.98525556
```

Open a terminal from your VM. Run the follow commands to import
airline performance data into your mongodb database.

```bash
cd ~/cisc_525/cisc-525-util/data/airline
mongoimport --db airline --collection planes --type csv --headerline --file ./plane-data.csv
mongoimport --db airline --collection carriers --type csv --headerline --file ./carriers.csv
mongoimport --db airline --collection airports --type csv --headerline --file ./airports.csv
```

You should see the output as follows:

```shelscript
kiet@student-VirtualBox:~/cisc_525/cisc-525-util/data/airline$ mongoimport --db airline --collection planes --type csv --headerline --file ./plane-data.csv
2020-04-11T15:13:03.821-0400	connected to: localhost
2020-04-11T15:13:03.881-0400	imported 5029 documents
kiet@student-VirtualBox:~/cisc_525/cisc-525-util/data/airline$ mongoimport --db airline --collection carriers --type csv --headerline --file ./carriers.csv
2020-04-11T15:13:03.889-0400	connected to: localhost
2020-04-11T15:13:03.906-0400	imported 1491 documents
kiet@student-VirtualBox:~/cisc_525/cisc-525-util/data/airline$ mongoimport --db airline --collection airports --type csv --headerline --file ./airports.csv
2020-04-11T15:48:34.888-0400	connected to: localhost
2020-04-11T15:48:34.931-0400	imported 3376 documents
```
Ignore the error message on extraneous column for now. 

Verify that you import your data correct as follows:

## Verify imported data  

In [49]:
# Verify the counts ...
print(airline.planes.find().collection.estimated_document_count(), 'plane types')
print(airline.carriers.find().collection.estimated_document_count(), 'carriers')
print(airline.airports.find().collection.estimated_document_count(), 'airports') 

5029 plane types
1491 carriers
3376 airports


## What are the attributes/fields for the airports, carriers, and planes collections?
### Plane data type:
- tailnum
- type
- manufacturer
- issue_date
- model
- status
- aircraft_type
- engine_type
- year

### Carrier:
- Code
- Description

### Airport:
- iata
- airport
- city
- state
- country
- lat
- long

In [50]:
airline.planes.find()[40]

{'_id': ObjectId('5e97c0d035850d690dc376d5'),
 'tailnum': 'N105UA',
 'type': 'Corporation',
 'manufacturer': 'BOEING',
 'issue_date': '10/15/2007',
 'model': '747-451',
 'status': 'Valid',
 'aircraft_type': 'Fixed Wing Multi-Engine',
 'engine_type': 'Turbo-Jet',
 'year': 1994}

In [51]:
airline.carriers.find()[2]

{'_id': ObjectId('5e97c13635850d690dc38a68'),
 'Code': '06Q',
 'Description': 'Master Top Linhas Aereas Ltd.'}

In [52]:
airline.airports.find()[0]

{'_id': ObjectId('5e97c14735850d690dc39041'),
 'iata': '00M',
 'airport': 'Thigpen ',
 'city': 'Bay Springs',
 'state': 'MS',
 'country': 'USA',
 'lat': 31.95376472,
 'long': -89.23450472}

## How many countries are in the airports collection
Attribute `country` 

In [6]:

airports = airline.airports.distinct('country')
print((airports), 'distinct countries')
# for airport in airports:
#     print(airport)

['USA', 'Thailand', 'Palau', 'N Mariana Islands', 'Federated States of Micronesia'] distinct countries


## How many plane manufacters are in the planes collection?
Attribute `manufacturer`

In [7]:
manufacturers = airline.planes.distinct('manufacturer')
print(len(manufacturers), 'manufacturers of airplanes')
# manufacturers
# for manufacturer in manufacturers:
#     print(manufacturer)

39 manufacturers of airplanes


## How many states are in the airports collections
Attribute `state`

In [8]:

states_and_territorries = airline.airports.distinct('state')
print(len(states_and_territorries), 'states and territorries')
# for state in states_and_territorries:
#     print(state)


57 states and territorries


## How many airports are in the state of HI?

In [9]:
hi_airports = list(airline.airports.find({'state':'HI'}))
# for hi_airport in hi_airports:
#     print(hi_airport['airport'])
print('**', len(hi_airports), 'airports in the State of Hawaii')    

** 16 airports in the State of Hawaii


 ## 1. How many plane types (use type) are there?

In [10]:
types = airline.planes.distinct('type')
print(len(types), 'plane types') 
#for types in planes

5 plane types


## 2. How many planes BOEING made over the years?

In [31]:
planes = list(airline.planes.find({'manufacturer':'BOEING'}))
print('**', len(planes), 'Planes Boeing made over the years')    

** 2061 Planes Boeing made over the years


## 3. How many aircraft types (use `aircraft_type`) are there?

In [391]:
aircraft_types = airline.planes.distinct('aircraft_type')
print( len(aircraft_types), 'Number of aircraft types')

4 Number of aircraft types


## 4.How many types of planes CESSNA (use manufacturer and aircaft_type, made over the years?

In [401]:
planes_at_CESSNA = list(airline.planes.find({'manufacturer':'CESSNA'},"aircraft_type"))
print( len(planes_at_CESSNA), 'types of planes made by CESSNA')  

12 types of planes made by CESSNA


## 5.List aircraft types made by each manufacturer

In [412]:
for aircrafts in airline.planes.find({"aircraft_type":{"$in":["Fixed Wing Multi-Engine","Fixed Wing Single-Engine","Rotorcraft","Balloon"]}},{"aircraft_type":1,"manufacturer":1,"_id":0}).limit(10):
    print(aircrafts)

{'manufacturer': 'EMBRAER', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'BOEING', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'BOEING', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'BOEING', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'EMBRAER', 'aircraft_type': 'Fixed Wing Multi-Engine'}
{'manufacturer': 'BOEING', 'aircraft_type': 'Fixed Wing Multi-Engine'}


## 6.List number of planes made by each manufacturer

In [203]:
for planes_by_manufacturers in airline.planes.aggregate([
    {"$group":{
        "_id": {"$toUpper": "$manufacturer"},
        "count": {"$sum": 1}
    }}   
]):
     print(planes_by_manufacturers)

{'_id': 'AERONCA', 'count': 1}
{'_id': 'COBB INTL/DBA ROTORWAY INTL IN', 'count': 1}
{'_id': 'MCDONNELL DOUGLAS AIRCRAFT CO', 'count': 105}
{'_id': 'BOEING OF CANADA LTD', 'count': 1}
{'_id': 'DIAMOND AIRCRAFT IND INC', 'count': 1}
{'_id': 'BAUMAN RANDY', 'count': 1}
{'_id': 'MARZ BARRY', 'count': 1}
{'_id': 'LEBLANC GLENN T', 'count': 1}
{'_id': 'FRIEDEMANN JON', 'count': 1}
{'_id': 'FREDERICK CHRIS K', 'count': 1}
{'_id': 'BELL', 'count': 1}
{'_id': 'WOBIG WAYNE R', 'count': 1}
{'_id': 'AIRBUS', 'count': 244}
{'_id': 'CANADAIR', 'count': 86}
{'_id': 'DOUGLAS', 'count': 36}
{'_id': 'SAAB-SCANIA', 'count': 28}
{'_id': 'BOMBARDIER INC', 'count': 547}
{'_id': 'STINNETT RON', 'count': 1}
{'_id': 'MCDONNELL DOUGLAS', 'count': 253}
{'_id': 'DEHAVILLAND', 'count': 17}
{'_id': 'SIKORSKY', 'count': 3}
{'_id': '', 'count': 549}
{'_id': 'AIRBUS INDUSTRIE', 'count': 434}
{'_id': 'MAULE', 'count': 2}
{'_id': 'EMBRAER', 'count': 588}
{'_id': 'BOEING', 'count': 2061}
{'_id': 'HELIO', 'count': 1}
{'_

## 7.Which manufacturer makes the least planes?

In [406]:
for least_planes_by_manufacturers in airline.planes.aggregate([
    {
        "$group":{
        "_id": {"$toUpper":"$manufacturer"},
        "count": {"$sum": 1},
            
    }} ,
    {
    "$sort": {"count": 1}
    },
    {"$limit":1}
]):
     print((least_planes_by_manufacturers))

{'_id': 'AERONCA', 'count': 1}


 ## 8.Which manufacturer makes the `most` planes?

In [338]:
for most_planes_by_manufacturers in airline.planes.aggregate([
    {
        "$group":{
        "_id": {"$toUpper":"$manufacturer"},
        "count": {"$sum": 1},
            
    }} ,
    {
    "$sort": {"count": -1}
    },
    {"$limit":1}
]):
     print((most_planes_by_manufacturers))

{'_id': 'BOEING', 'count': 2061}


 ## 9.Which state has the `most` airports?

In [335]:
for most_states_as_airports in airline.airports.aggregate([
    {"$group":
     {"_id": {"$toUpper": "$state"},
        "count": {"$sum": 1}
    }},
    {
    "$sort": {"count": -1}
    },
    {"$limit":1}
]):
     print((most_states_as_airports))

{'_id': 'AK', 'count': 263}


## 10.Which state has the least airports?

In [402]:
for least_states_as_airports in airline.airports.aggregate([
    {"$group":
     {"_id": {"$toUpper": "$state"},
        "count": {"$sum": 1}
    }},
    {
    "$sort": {"count": 1}
    },
    {"$limit":1}
]):
     print((least_states_as_airports))

{'_id': 'GU', 'count': 1}


## 11.Which state has the second largest number of airports?

In [ ]:
for second_largest_states_as_airports in airline.airports.aggregate([
    {"$group":
     {"_id": {"$toUpper": "$state"},
        "count": {"$sum": 1}
    }},
    {
    "$project": {
        max_value: 1, 
        min_value: 1, 
        second_max_value: {"$arrayElemAt": ['$all_value', 1]}
    }
    },
    
]):
     print((second_largest_states_as_airports))

## 12.List all the years in the planes collection. Exclude (0 and 'None')

In [ ]:
for most_planes_per_year in airline.planes.aggregate([
    {"$group":
     {"_id": {"year": "$year", "$ne": null},
        "count": {"$sum": 1}
    }},  
]):
     print((most_planes_per_year))

## 13. Which year produced the most planes?

In [410]:
for most_planes_per_year in airline.planes.aggregate([
    {"$group":
     {"_id": {"year": "$year"},
        "count": {"$sum": 1}
    }},
    {
    "$sort": {"count": -1}
    },
    {"$limit":1}
]):
     print((most_planes_per_year))

{'_id': {'year': None}, 'count': 549}


## 14.Which year produced the `least` planes?

In [409]:
for least_planes_per_year in airline.planes.aggregate([
    {"$group":
     {"_id": {"year": "$year"},
        "count": {"$sum": 1}
    }},
    {
    "$sort": {"count": 1}
    },
    {"$limit":1}
]):
     print((least_planes_per_year))

{'_id': {'year': 1964}, 'count': 1}


## 15. List number of planes by aircraft types


In [362]:
for planes_by_aircraft_type in airline.planes.aggregate([
    {"$group":{
        "_id": {"$toUpper": "$aircraft_type"},
        "count": {"$sum": 1}
    }}   
]):
     print((planes_by_aircraft_type))

{'_id': 'BALLOON', 'count': 3}
{'_id': 'FIXED WING MULTI-ENGINE', 'count': 4440}
{'_id': '', 'count': 549}
{'_id': 'FIXED WING SINGLE-ENGINE', 'count': 29}
{'_id': 'ROTORCRAFT', 'count': 8}


## 16. List engine types made by each manufacturer



In [387]:
for planes in airline.planes.find({"engine_type":{"$in":["Turbo-Fan","Turbo-Jet","Reciprocating","Turbo-Prop","Turbo-Shaft","4 Cycle" ]}},{"engine_type":1,"manufacturer":1,"_id":0}):
    print(planes)

{'manufacturer': 'EMBRAER', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'EMBRAER', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'EMBRAER', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'EMBRAER',

{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'MCDONNELL DOUGLAS', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'CANADAIR', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'MCDONNELL DOUGLAS', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'MCDONNELL DOUGLAS', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS INDUSTRIE', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'CESSNA', 'engine_type': '4 Cycle'}
{'manufacturer': 'AIRBUS INDUSTRI

{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'AIRBUS', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'DOUGLAS', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'MCDONNELL DOUGLAS AIRCRAFT CO', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'CANADAIR', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'AIRBUS', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'DOUGLAS', 'engine_type': 'Turbo-Jet'}
{'manufacturer': 'BOMBARDIER INC', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'BOEING', 'engine_type': 'Turbo-Fan'}
{'manufacturer': 'EMBRAER', 'engine_type': 'Turbo-Fan

## 17. List planes made by aircraft types

In [365]:
for planes_by_aircraft_type in airline.planes.aggregate([
    {"$group":{
        "_id": {"$toUpper": "$aircraft_type"},
       
    }}   
]):
     print((planes_by_aircraft_type))

{'_id': 'BALLOON'}
{'_id': 'FIXED WING MULTI-ENGINE'}
{'_id': ''}
{'_id': 'FIXED WING SINGLE-ENGINE'}
{'_id': 'ROTORCRAFT'}


## 18. List planes made by engine type

In [388]:
for planes_by_engine_type in airline.planes.aggregate([
    {"$group":{
        "_id": {"$toUpper": "$engine_type"},
       
    }}   
]):
     print((planes_by_engine_type))

{'_id': '4 CYCLE'}
{'_id': 'TURBO-SHAFT'}
{'_id': 'TURBO-PROP'}
{'_id': ''}
{'_id': 'NONE'}
{'_id': 'TURBO-FAN'}
{'_id': 'TURBO-JET'}
{'_id': 'RECIPROCATING'}


## Assignment
Answer the following questions:

1. How many plane types (use `type`) are there?
2. How many planes `BOEING` made over the years?
3. How many aircraft types (use `aircraft_type`) are there?
4. How many types of planes `CESSNA` (use `manufacturer` and `aircaft_type`, made over the years?
5. List aircraft types made by each manufacturer
6. List number of planes make by each manufacturer
7. Which manufacturer makes the `least` planes?
8. Which manufacturer makes the `most` planes?
9. Which state has the `most` airports?
10. Which state has the `least` airports?
11. Which state has the `second` largest number of airports?
12. List all the years in the `planes` collection. Exclude (0 and 'None')
13. Which year produced the `most` planes?
14. Which year produced the `least` planes?
15. List number of planes by aircraft types
16. List engine types made by each manufacturer
17. List planes made by aircraft types
18. List planes made by engine type